In [1]:
import pandas as pd
import requests
import os
import io
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
import json
import numpy as np
from PIL import Image
import arabic_reshaper 
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime, date, time
from pandas import Grouper
from matplotlib import pyplot
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
import xlsxwriter


In [11]:
df=pd.read_excel('Timesheet.xlsx')

In [12]:
df.sample(3)

,Month,Iqama,Name,Working Days,Project_Fees,Basic Salary,Additions,Discounts,الجنسية
2,31,2571459912,Waseem Akram Shamraiz,31,2800,1200,0,0,2
1,31,2572404644,Muhammad Ateeq Abdul Latif,31,2800,1200,0,0,2
0,31,2571459698,Farhad Ahmed Masood Ur Rehman,31,2800,1200,0,0,2


In [13]:
df['راتب اساسي'] = 400
df['بدل سكن']  =67
df['بدلات اخري'] =df['Basic Salary'] - df['بدل سكن'] - df['راتب اساسي']
df['تذكرة السفر']  = 3000/720 * df['Working Days']
df['كارت البنك'] = 7.5
df['تحويل البنك'] = 7
df['خروج وعودة'] = 200/720*df['Working Days']
df['التامينات الاجتماعية'] = 2/100*df['Basic Salary']
df['نهاية الخدمة'] = df['Basic Salary']/2/360*df['Working Days']
df['اقامة'] = 650/354*df['Working Days']
df['التامين الطبي'] = 390/360 *df['Working Days']
df['فحص الطبي'] = 100/720 *df['Working Days']
df['كارت العمل'] = 9700/354 *df['Working Days']
df['بدل الاجازة'] = df['Basic Salary']/30*21/360*df['Working Days']
df['التاشيرة'] =  2000/720 * df['Working Days']
working_days = df['Working Days']
Nat = df['الجنسية']
net_business = []
Agent_Commission = []


for items in working_days:
    if items > 30 :
        net_business.append(30)
    else :
        net_business.append(items)

        
df['net_business']= net_business   


for agents in Nat :
    if agents < 2 :
        Agent_Commission.append(1800/720*df['Working Days'][agents-1])
    else :
        Agent_Commission.append(0)

    

df['رسوم مكتب'] = Agent_Commission
df['القيمة الاجارية'] = df['Project_Fees'] / 30 * df['net_business']
df['الراتب'] = df['Basic Salary'] /30 * df['net_business']
df['fees'] = (df['Project_Fees'] - df['Basic Salary']) /30 * df['net_business']
df['الرسوم الادارية'] =df['fees']-(df['التاشيرة']+df['بدل الاجازة']+df['كارت العمل']+df['فحص الطبي']+df['التامين الطبي']+df['اقامة']+df['نهاية الخدمة']+df['التامينات الاجتماعية']+df['خروج وعودة']+df['تحويل البنك']+df['كارت البنك']+df['تذكرة السفر']+df['رسوم مكتب'])
df['اجمالي الراتب'] =df['الراتب'] + df['Additions'] - df['Discounts']
df['خاضع للضريبة'] = df['الرسوم الادارية'] + df['فحص الطبي'] + df['التامين الطبي'] + df['تحويل البنك'] +df['كارت البنك']
df['غير خاضع'] = df['fees'] - df['خاضع للضريبة']



for i in df['الرسوم الادارية'] :
    if i < 0 :
        df['الضريبة']=(df['فحص الطبي'] + df['التامين الطبي'] + df['تحويل البنك'] +df['كارت البنك'])*15/100
    else :
        df['الضريبة'] = df['خاضع للضريبة'] *15/100



df['Total_Without_Vat'] = df['غير خاضع']+df['خاضع للضريبة']+df['اجمالي الراتب']
df['Total'] = df['Total_Without_Vat'] + df['الضريبة']
Invoice = df [['Name','Iqama','Working Days','net_business','اجمالي الراتب','fees','Total_Without_Vat','الضريبة','Total']]
CreateInvoice=pd.ExcelWriter('Invoice.xlsx')
Invoice.to_excel(CreateInvoice,index=False)
CreateInvoice.close()
CreateInvoiceDetails =pd.ExcelWriter('Invoice Details.xlsx')
df.to_excel(CreateInvoiceDetails,index=False)
CreateInvoiceDetails.close()


In [14]:
df['خاضع للضريبة'].sum()*15/100

np.float64(144.78983050847455)

0    30
Name: net_business, dtype: int64